# Starting

In [47]:
using DelimitedFiles
using CUDA

In [48]:
R_Agg = 9
r_max = 3.5

3.5

# Inizializating Variables

In [50]:
X = Float64.(readdlm("../../../data/init/Sphere/$R_Agg.xyz")[3:end,2:end]) |> cu

dists  = CUDA.zeros(size(X, 1), size(X, 1)) |> cu
idx    = CUDA.zeros(Int, size(X, 1), size(X, 1)) |> cu

force = CUDA.zeros(size(dists));

LoadError: CUDA error: an illegal instruction was encountered (code 715, ERROR_ILLEGAL_INSTRUCTION)

# Functions

In [46]:
function cuda_distance_kernel!(idx, points ,r_max)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    if i <= size(points, 1) && j <= size(points, 1) 
        # d = 0.0f0
        d = sqrt(
            (points[i, 1] - points[j, 1])^2 + 
            (points[i, 2] - points[j, 2])^2 + 
            (points[i, 3] - points[j, 3])^2
            )
        idx[i, j] = d
        # if d < r_max
        #     idx[i, j] = i
        #     # dists[i, j] = d
        # else
        #     idx[i, j] = 0
        #     # dists[i, j] = 0
        # end 
    end
    return nothing
end

function dist_kernel!(points, idx, dist)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    if i <= size(idx, 1) && j <= size(idx, 2) 
        if idx[i,j] == 0
            dist[i,j] = 0.0
        else
            dist[i,j] = sqrt(
                (points[idx[i,j],1] - points[i,1])^2 + 
                (points[idx[i,j],2] - points[j,2])^2 + 
                (points[idx[i,j],3] - points[j,3])^2
                )
        end
    end
    return nothing
end

dist_kernel! (generic function with 1 method)

# idx

In [45]:
threads = (32, 32)
blocks = (div(size(X,1),threads[1])+1, div(size(X,1),threads[1])+1)

CUDA.@time @cuda threads=threads blocks=blocks cuda_distance_kernel!(idx, X, r_max)
idx

ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.
ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2

LoadError: CUDA error: an illegal instruction was encountered (code 715, ERROR_ILLEGAL_INSTRUCTION)

# Force

In [ ]:
threads = (32, 32)
blocks = (div(size(X,1),threads[1])+1, div(size(X,1),threads[1])+1)

CUDA.@time @cuda threads=threads blocks=blocks force_kernel!(X, idx, dist)
dist

LoadError: UndefVarError: dist not defined